# **Unsupervised Machine Learning for Geospatial Data Analysis**


## Imports and Setup
### Install libraries
First, install any additional libraries that are not installed by default (e.g., rasterio, earthpy)..

In [ ]:
# Install rasterio and earthpy libraries
!pip install rasterio
!pip install earthpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.7 MB/s eta 0:00:00


### Import libraries
Import the necessary libraries (pandas, numpy, scikit-learn, rasterio, etc.).

In [ ]:
# Import libraries
import rasterio
import earthpy.plot as ep
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture

### Mount Google Drive
Next, mount your Google Drive. You will be prompted to authorize access to your Google Drive. Once mounted, you can read/write files in /content/drive/MyDrive.

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


### Define paths and variables
Define the the paths to access your own directory structure in Google Drive. In this tutorial, we use:
-A CSV training dataset (Bul_TrainingData_2024.csv) containing pixel values and their corresponding classes.
- A multiband Sentinel-2 image (Bul_S2_2024.tif).
- PALSAR ScanSAR polarization

In [ ]:
# Define path that contains the datasets
S2_Image_Path = '/content/drive/MyDrive/Bulawayo_Dataset_2024/Bul_S2_2024.tif'
Palsar_Image_Path = '/content/drive/MyDrive/Bulawayo_Dataset_2024/Bul_Palsar_HV_2024.tif'

## Load Images
We will use rasterio to open the .tif file. These are Sentinel-2 imagery and ALOS PALSAR ScanSAR HV polarization.

In [ ]:
# Load Sentinel-2 image
with rasterio.open(S2_Image_Path) as src_s2:
    s2_array = src_s2.read()  # shape: [bands, height, width]
    profile = src_s2.profile
    height, width = src_s2.height, src_s2.width

# Load PALSAR HV image
with rasterio.open(Palsar_Image_Path) as src_palsar:
    palsar_array = src_palsar.read(1)  # HV is single-band: [height, width]

# Ensure shapes match
assert s2_array.shape[1:] == palsar_array.shape, "Image sizes don't match. Reproject or resample needed."

### Display images
Display the Sentinel-2 composite and the PALSAR ScanSAR HV polorization image.

In [ ]:
# Select bands for RGB composite: B4 (Red), B3 (Green), B2 (Blue)
# Note: Check your band ordering if uncertain
red = s2_array[2, :, :]  # B4
green = s2_array[1, :, :]  # B3
blue = s2_array[0, :, :]  # B2

# Stack and normalize for RGB display
rgb = np.stack([red, green, blue], axis=-1)
rgb_min, rgb_max = 0, 0.3  # Adjust depending on your scaling
rgb_display = np.clip((rgb - rgb_min) / (rgb_max - rgb_min), 0, 1)

# Normalized HV (grayscale display)
hv_min, hv_max = 0,1
hv_display = np.clip((palsar_array - hv_min) / (hv_max - hv_min), 0, 1)

# Plot side by side
fig, axs = plt.subplots(1, 2, figsize=(14, 7))

axs[0].imshow(rgb_display)
axs[0].set_title('Sentinel-2 RGB (B4, B3, B2)')
axs[0].axis('off')

axs[1].imshow(hv_display, cmap='gray')
axs[1].set_title('PALSAR HV Backscatter')
axs[1].axis('off')

plt.tight_layout()
plt.show()

## Prepare the Feature Array

Combine Sentinel-2 RGB bands and PALSAR HV into one feature set.

In [ ]:
# Flatten the bands
r = red.flatten()
g = green.flatten()
b = blue.flatten()
hv = palsar_array.flatten()

# Stack features: [R, G, B, HV]
features = np.stack([r, g, b, hv], axis=1)

# Remove rows with NaNs
valid_mask = ~np.isnan(features).any(axis=1)
features_clean = features[valid_mask]

# Normalize
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_clean)

## Clustering Techniques
### Kmeans clustering
K-Means clustering is an unsupervised machine learning algorithm that partitions data into a predefined number of clusters (k) by minimizing the distance between data points and their respective cluster centers. In geospatial analysis, it groups pixels with similar spectral characteristics — useful for land cover classification without labeled data.

Next, we apply unsupervised K-Means clustering to pixel-level feature vectors extracted from the input imagery. The resulting cluster labels are reshaped to match the original image dimensions, allowing us to generate a land cover classification map. Each pixel is assigned to a cluster based on spectral similarity, which may correspond to different land cover types such as vegetation, urban areas, water bodies, or bare soil.

In [ ]:
# Apply KMeans clustering with 5 clusters
# `random_state` ensures reproducibility (same results every run)
kmeans = KMeans(n_clusters=5, random_state=42)

# Fit the model to the scaled features and get cluster labels
# features_scaled: shape [n_valid_pixels, n_features]
kmeans_labels = kmeans.fit_predict(features_scaled)

# Create an empty array with the same size as the full image (1D), initialized to -1
# r is a flattened image band; we use its shape to ensure consistency
kmeans_map = np.full(r.shape, fill_value=-1)

# Insert the cluster labels into positions where the data was valid (no NaNs, etc.)
kmeans_map[valid_mask] = kmeans_labels

# Reshape the 1D array back into the original 2D image shape for mapping or export
kmeans_map = kmeans_map.reshape((height, width))

Next, display the K-Means clustered map.

In [ ]:
# Create a figure and axis for plotting
fig, ax = plt.subplots(figsize=(10, 8))  # Set the figure size

# Display the KMeans clustered map using a categorical colormap ('tab10')
im = ax.imshow(kmeans_map, cmap='tab10')

# Set the map title with larger font size
ax.set_title('Land Cover Clusters (KMeans)', fontsize=14)

# Remove axis ticks and labels for a clean map display
ax.axis('off')

# Create a colorbar axis next to the map using axes_grid1
from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0.05)  # Adjust size and padding

# Add a colorbar for cluster reference and label it
cbar = plt.colorbar(im, cax=cax)
cbar.set_label('Cluster')  # Label the colorbar

# Adjust layout to avoid overlapping elements
plt.tight_layout()

# Show the final map with the colorbar
plt.show()

### MiniBatch KMeans

MiniBatch K-Means is a faster and more memory-efficient version of the traditional K-Means clustering algorithm. Instead of using the entire dataset to compute cluster centroids in each iteration, it processes small random subsets (mini-batches) of the data. This reduces computation time and memory usage, making it ideal for large datasets like satellite images or high-resolution geospatial rasters.

Next, we perform unsupervised clustering using MiniBatch KMeans with 5 clusters and a batch size of 1000. We fit the model on the features_scaled dataset, which contains pixel-level features such as spectral bands or backscatter values. After fitting, the code returns a cluster label for each valid pixel. The labels are then placed back into a full-size array (mb_kmeans_map) using a valid pixel mask to ensure spatial consistency. Finally, the 1D array is reshaped into a 2D format (height × width) to reconstruct a classified land cover map, where each pixel is assigned a cluster based on similarity in spectral or SAR features.

In [ ]:
# Apply MiniBatch KMeans clustering with 5 clusters
# Uses small batches (size = 1000) for faster computation on large datasets
mb_kmeans = MiniBatchKMeans(n_clusters=5, batch_size=1000, random_state=42)

# Fit the model and predict cluster labels for each valid pixel
mb_labels = mb_kmeans.fit_predict(features_scaled)

# Create a placeholder array with the shape of the original image (flattened)
# Initialize with -1 for pixels that will not be classified
mb_kmeans_map = np.full(r.shape, fill_value=-1)

# Insert the cluster labels at valid pixel locations
mb_kmeans_map[valid_mask] = mb_labels

# Reshape the 1D classified map back into 2D image format
mb_kmeans_map = mb_kmeans_map.reshape((height, width))

Next, display the MiniBatch KMeans clustered map.

In [ ]:
# Create a figure and axis object for plotting
fig, ax = plt.subplots(figsize=(10, 8))  # Set figure size (width, height) in inches

# Display the MiniBatch KMeans cluster map using the 'tab10' colormap
im = ax.imshow(mb_kmeans_map, cmap='tab10')

# Add a title to the map with increased font size
ax.set_title('Land Cover Clusters (MB KMeans)', fontsize=14)

# Remove axis ticks and labels for a clean visual map
ax.axis('off')

# Create a divider to add a colorbar next to the map
from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)

# Append a new axes (cax) to the right of the map for the colorbar
cax = divider.append_axes("right", size="3%", pad=0.05)

# Add and customize the colorbar for interpreting cluster values
cbar = plt.colorbar(im, cax=cax)
cbar.set_label('Cluster')  # Label for the colorbar

# Automatically adjust spacing to prevent overlap of elements
plt.tight_layout()

# Display the final plot
plt.show()

### Gaussian Mixture Model (GMM)

A Gaussian Mixture Model (GMM) is a probabilistic clustering algorithm that assumes the data is generated from a mixture of several Gaussian distributions, each representing a different cluster. Unlike KMeans, which assigns data points to clusters based solely on distance to a centroid, GMM models the probability of each point belonging to a cluster and allows for soft classification (points can belong to multiple clusters with varying probabilities). It also accounts for cluster shapes, orientations, and variances, making it more flexible for modeling real-world data distributions.

Next, we create a GMM with 5 components (clusters) and a shared covariance structure (covariance_type='tied'). The model is fitted to the standardized geospatial feature vectors (features_scaled), which may include spectral bands and radar data. Once trained, it assigns a cluster label to each valid pixel. These labels are stored in a 1D array (gmm_labels) and then mapped back to their corresponding pixel positions using the valid_mask. The final map (gmm_map) is reshaped into the original image dimensions (height × width), producing a spatially explicit land cover classification map based on GMM clustering.

In [ ]:
# Initialize the Gaussian Mixture Model with 5 components
# 'tied' means all clusters share the same covariance matrix (efficient and stable)
gmm = GaussianMixture(n_components=5, covariance_type='tied', random_state=42)

# Fit the model to the scaled pixel features and predict cluster assignments
gmm_labels = gmm.fit_predict(features_scaled)

# Create an empty array the same size as the full image (flattened), filled with -1
# This ensures unclassified (invalid) pixels remain marked
gmm_map = np.full(r.shape, fill_value=-1)

# Assign cluster labels only to valid pixels (e.g., non-NaN)
gmm_map[valid_mask] = gmm_labels

# Reshape the 1D map into a 2D image for display or export
gmm_map = gmm_map.reshape((height, width))

Next, display the Gaussian Mixture Model (GMM) clustered map.

In [ ]:
# Create a figure and axis for plotting the land cover map
fig, ax = plt.subplots(figsize=(10, 8))  # Set figure size in inches (width, height)

# Display the GMM clustering result as an image using a categorical colormap ('tab10')
im = ax.imshow(gmm_map, cmap='tab10')

# Add a title to the map with increased font size
ax.set_title('Land Cover Map (GMM)', fontsize=14)

# Turn off axis ticks and labels for a cleaner visual presentation
ax.axis('off')

# Create a divider to append a colorbar axis next to the plot
from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)

# Append a new axis to the right side for the colorbar
cax = divider.append_axes("right", size="3%", pad=0.05)

# Create and display the colorbar linked to the image, showing cluster IDs
cbar = plt.colorbar(im, cax=cax)
cbar.set_label('Cluster')  # Add a label to the colorbar

# Automatically adjust layout spacing to prevent overlap
plt.tight_layout()

# Display the final map with the colorbar
plt.show()

### Display clustered maps
Next, display all the K-Means, MiniBatch K-Means, and GMM clustered maps.

In [ ]:
# Visualization
fig, axs = plt.subplots(1, 3, figsize=(20, 7))

# KMeans Map
im0 = axs[0].imshow(kmeans_map, cmap='tab10')
axs[0].set_title('KMeans Clustering')
axs[0].axis('off')

# MiniBatch KMeans Map
im1 = axs[1].imshow(mb_kmeans_map, cmap='tab10')
axs[1].set_title('MiniBatch KMeans Clustering')
axs[1].axis('off')

# GMM Map
im2 = axs[2].imshow(gmm_map, cmap='tab10')
axs[2].set_title('Gaussian Mixture Model (GMM)')
axs[2].axis('off')

plt.tight_layout()
plt.show()

### Assign class names
Next, assign class names to the K-Means clustered map.

In [ ]:
# Class name mapping
class_names = {
    0: 'Cropland',
    1: 'Dense Vegetation',
    2: 'Sparse Vegetation',
    3: 'Built-up',
    4: 'Bare Soil'
}

# Color mapping (can be hex or named colors)
class_colors = {
    0: '#FFFF00',   # yellow
    1: '#2ca02c',   # dark green
    2: '#98df8a',   # light green
    3: '#d62728',   # red
    4: '#8c564b'    # brown
}

# Create color list sorted by cluster ID
color_list = [class_colors[i] for i in sorted(class_colors)]
custom_cmap = mcolors.ListedColormap(color_list)

# Create map
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(kmeans_map, cmap=custom_cmap, vmin=0, vmax=4)
ax.set_title('KMeans Land Cover', fontsize=14)
ax.axis('off')

# Create custom legend
legend_handles = [
    mpatches.Patch(color=class_colors[i], label=class_names[i])
    for i in sorted(class_names)
]
plt.legend(handles=legend_handles, loc='lower center', bbox_to_anchor=(0.5, -0.1),
           ncol=3, frameon=False)

plt.tight_layout()
plt.show()

## Export the Clustered Land Cover Maps
We can save the land cover maps to a new GeoTIFF using rasterio and export to Google Drive.

In [ ]:
# Save clustered land cover map
# Helper function to export a map
def export_cluster_map(map_array, output_path, profile):
    with rasterio.open(
        output_path, 'w',
        driver='GTiff',
        height=map_array.shape[0],
        width=map_array.shape[1],
        count=1,
        dtype=np.uint8,
        crs=profile['crs'],
        transform=profile['transform']
    ) as dst:
        dst.write(map_array.astype(np.uint8), 1)

# Paths to save the maps
kmeans_path = '/content/drive/MyDrive/Bulawayo_Dataset_2024/Bul_LC_KMeans_2024.tif'
mbkmeans_path = '/content/drive/MyDrive/Bulawayo_Dataset_2024/Bul_LC_MBKMeans_2024.tif'
gmm_path = '/content/drive/MyDrive/Bulawayo_Dataset_2024/Bul_LC_GMM_2024.tif'

# Export each clustered map
export_cluster_map(kmeans_map, kmeans_path, profile)
export_cluster_map(mb_kmeans_map, mbkmeans_path, profile)
export_cluster_map(gmm_map, gmm_path, profile)

print("All cluster maps exported as GeoTIFFs to Google Drive.")